In [1]:
# Imports
import numpy as np
import pandas as pd
import pulp

from warnings import filterwarnings
filterwarnings("ignore")


vehicles_df = pd.read_csv('../data/Master_data/vehicle_master_data.csv')
distances_df = pd.read_csv('../data/Master_data/distance_master_data.csv')

In [2]:
input_df = pd.read_csv("../data/Input_data/Input.csv")

In [3]:
vehicles = {m:[{i:vehicles_df[i][j] for i in (vehicles_df[vehicles_df['Transportation modes']==m].columns[1:])} for j in range(len(vehicles_df[vehicles_df['Transportation modes']==m]))] for m in vehicles_df['Transportation modes']}
distances = {(distances_df['Source'][j], distances_df['Destination'][j]):{i:distances_df[i][j] for i in distances_df.columns[2:]} for j in range(len(distances_df))}

df = distances_df.join(vehicles_df, how="cross")
df['decisions'] = df['Source'] + ', ' + df['Destination'] + ', ' + df['Transportation modes'] + ', ' + df['Vehicle_type']

In [4]:
master_availability = dict(zip(df['Vehicle_type'].unique(), (1,)*len(df['Vehicle_type'].unique())))

master_demand = {}
for i, j in zip(np.array(df[['Source', 'Destination']].drop_duplicates()), [0,]*len(df[['Source', 'Destination']].drop_duplicates())):
    master_demand[tuple(i)] = j

In [5]:
demand = {}
for i, j in zip(np.array(input_df[['From', 'To']]), input_df['Quantity (MT)']):
    demand[tuple(i)] = j
    
for k in master_demand.keys():
    if k not in demand.keys():
        demand[k] = 0

In [6]:
%%time

##########################################################
#------------------- Model Definition -------------------#
##########################################################
# Create a LP minimization problem
model = pulp.LpProblem("Co2e Optimization", pulp.LpMinimize)
            
############################################################
#------------------- Decision Variables -------------------#
############################################################

choice_decision =  pulp.LpVariable.dicts("Choice_Decision_", (tuple(df['decisions'][i].split(", ")) + (j,)
                      for i in range(len(df)) for j in range(master_availability[df['decisions'][i].split(", ")[-1]])),
                                  lowBound=0, cat="Binary")

############################################################
#------------------- Objective Function -------------------#
############################################################

obj_tup = [choice_decision[(s, d, m, v, c)]\
    *df[(df['Source']==s) & (df['Destination']==d) & (df['Transportation modes']==m) & (df['Vehicle_type']==v)][m+' Distance (km)'].item()\
    *df[(df['Source']==s) & (df['Destination']==d) & (df['Transportation modes']==m) & (df['Vehicle_type']==v)]['Co2e (g/km)'].item()/1000
          for (s, d, m, v, c) in choice_decision.keys()]
model += pulp.lpSum(obj_tup)


#####################################################
#------------------- Constraints -------------------#
#####################################################
        
(s0, d0, m0, v0, c0) = list(choice_decision.keys())[0]
col = []

for (s, d, m, v, c) in choice_decision.keys():
    if(s0, d0) == (s, d):
        col.append(choice_decision[(s, d, m, v, c)]*df[(df['Source']==s) & (df['Destination']==d) & (df['Transportation modes']==m) & (df['Vehicle_type']==v)]['Capacity (metric tons)'].item())
    else:
        if demand[(s0, d0)] == 0:
            model += (pulp.lpSum(col)==demand[(s0, d0)])
            (s0, d0) = (s, d)
            col=[]
            col.append(choice_decision[(s, d, m, v, c)]*df[(df['Source']==s) & (df['Destination']==d) & (df['Transportation modes']==m) & (df['Vehicle_type']==v)]['Capacity (metric tons)'].item())
        elif demand[(s0, d0)] >= 0:
            model += (pulp.lpSum(col)>=demand[(s0, d0)])
            (s0, d0) = (s, d)
            col=[]
            col.append(choice_decision[(s, d, m, v, c)]*df[(df['Source']==s) & (df['Destination']==d) & (df['Transportation modes']==m) & (df['Vehicle_type']==v)]['Capacity (metric tons)'].item())

if demand[(s0, d0)] == 0:
    model += (pulp.lpSum(col)==demand[(s0, d0)])
elif demand[(s0, d0)] >= 0:
    model += (pulp.lpSum(col)>=demand[(s0, d0)])

CPU times: total: 812 ms
Wall time: 805 ms


In [7]:
%%time
###############################################
#------------------- Solve -------------------#
###############################################

model.solve()
print(pulp.LpStatus[model.status])

Optimal
CPU times: total: 31.2 ms
Wall time: 164 ms


In [8]:
#######################################################################
#------------------- Mappings Necessary for Output -------------------#
#######################################################################

mt = []
for v in model.variables():
    if  v.varValue != 0:
        mt.append(v.name.split('__(')[1].rstrip(')').split(',_') + [v.varValue,])
        
mt_output = pd.DataFrame(mt, columns=['Source', 'Destination', 'Transportation mode', 'Vehicle type', 'Vehicle number', 'Count'])
mt_output_df = mt_output.applymap(lambda x: x.strip("'") if isinstance(x, str) else x)
mt_output_df['S_D_M'] = mt_output_df['Source'] + '_' + mt_output_df['Destination'] + '_' + mt_output_df['Transportation mode']

dist_list = []
vals = []
for c in distances_df.columns[2:]:
    for i in range(len(distances_df)):
        dist_list.append(distances_df['Source'][i] + '_' + distances_df['Destination'][i] + '_' + c.split(' ')[0])
        vals.append(distances_df[c][i])
        
distance_mappings = dict(zip(dist_list, vals))
quantity_mappings = dict(zip(vehicles_df['Vehicle_type'], vehicles_df['Capacity (metric tons)']))
emission_mappings = dict(zip(vehicles_df['Vehicle_type'], vehicles_df['Co2e (g/km)']))

mt_output_df['Loaded quantity'] = mt_output_df['Vehicle type'].str.replace('_', ' ').map(quantity_mappings)
mt_output_df['Distance'] = mt_output_df['S_D_M'].map(distance_mappings)
mt_output_df['Emission'] = (mt_output_df['Vehicle type'].str.replace('_', ' ').map(emission_mappings))*mt_output_df['Distance']/1000

In [9]:
###########################################################
#------------------- Generating Output -------------------#
###########################################################

vehicle_count = pd.pivot_table(mt_output_df, values='Count', index=['Source', 'Destination'],
                       columns=['Vehicle type'], aggfunc='sum').reset_index()

vehicle_count['Total Vehicles in Use'] = vehicle_count.iloc[:, 2:].sum(axis=1)


material_transport_quantity = pd.pivot_table(mt_output_df, values='Loaded quantity', index=['Source', 'Destination'],
                       columns=['Vehicle type'], aggfunc='sum').reset_index()

material_transport_quantity['Total Quantity Transported'] = material_transport_quantity.iloc[:, 2:].sum(axis=1)


co2_emission = pd.pivot_table(mt_output_df, values='Emission', index=['Source', 'Destination'],
                       columns=['Vehicle type'], aggfunc='sum').reset_index()

co2_emission['Total co2 Emission (kg)'] = co2_emission.iloc[:, 2:].sum(axis=1)

vehicle_count.fillna(0, inplace=True)
material_transport_quantity.fillna(0, inplace=True)
co2_emission.fillna(0, inplace=True)

In [10]:
vehicle_count

Vehicle type,Source,Destination,CARGO_PLANE,GOODS_RAIL,HGV_(Large),MIXED_RAIL,Total Vehicles in Use
0,Bangalore,Chennai,0.0,1.0,0.0,0.0,1.0
1,Bangalore,Kolkata,1.0,1.0,1.0,1.0,4.0
2,Bangalore,Mumbai,0.0,1.0,0.0,0.0,1.0
3,Chennai,Bangalore,0.0,0.0,0.0,1.0,1.0
4,Chennai,Delhi,0.0,1.0,0.0,0.0,1.0
5,Chennai,Kolkata,0.0,0.0,0.0,1.0,1.0
6,Chennai,Mumbai,0.0,1.0,0.0,0.0,1.0
7,Delhi,Chennai,0.0,1.0,0.0,0.0,1.0
8,Delhi,Kolkata,0.0,1.0,0.0,1.0,2.0
9,Delhi,Mumbai,0.0,1.0,0.0,0.0,1.0


In [11]:
material_transport_quantity

Vehicle type,Source,Destination,CARGO_PLANE,GOODS_RAIL,HGV_(Large),MIXED_RAIL,Total Quantity Transported
0,Bangalore,Chennai,0.0,5000.0,0.0,0.0,5000.0
1,Bangalore,Kolkata,100.0,5000.0,36.0,8000.0,13136.0
2,Bangalore,Mumbai,0.0,5000.0,0.0,0.0,5000.0
3,Chennai,Bangalore,0.0,0.0,0.0,8000.0,8000.0
4,Chennai,Delhi,0.0,5000.0,0.0,0.0,5000.0
5,Chennai,Kolkata,0.0,0.0,0.0,8000.0,8000.0
6,Chennai,Mumbai,0.0,5000.0,0.0,0.0,5000.0
7,Delhi,Chennai,0.0,5000.0,0.0,0.0,5000.0
8,Delhi,Kolkata,0.0,5000.0,0.0,8000.0,13000.0
9,Delhi,Mumbai,0.0,5000.0,0.0,0.0,5000.0


In [12]:
co2_emission

Vehicle type,Source,Destination,CARGO_PLANE,GOODS_RAIL,HGV_(Large),MIXED_RAIL,Total co2 Emission (kg)
0,Bangalore,Chennai,0.000000,29.621987,0.000000,0.000000,29.621987
1,Bangalore,Kolkata,859.627824,158.233376,302.963544,173.680592,1494.505335
2,Bangalore,Mumbai,0.000000,85.563356,0.000000,0.000000,85.563356
3,Chennai,Bangalore,0.000000,0.000000,0.000000,32.513774,32.513774
4,Chennai,Delhi,0.000000,177.737777,0.000000,0.000000,177.737777
5,Chennai,Kolkata,0.000000,0.000000,0.000000,151.053421,151.053421
6,Chennai,Mumbai,0.000000,104.754571,0.000000,0.000000,104.754571
7,Delhi,Chennai,0.000000,177.737777,0.000000,0.000000,177.737777
8,Delhi,Kolkata,0.000000,132.834041,0.000000,145.801698,278.635739
9,Delhi,Mumbai,0.000000,116.130756,0.000000,0.000000,116.130756


In [13]:
vehicle_count.to_csv('../data/Output_data/v1/Vehicle_Count.csv', index=False)
material_transport_quantity.to_csv('../data/Output_data/v1/Material_Transport_Quantity.csv', index=False)
co2_emission.to_csv('../data/Output_data/v1/CO2_Emission.csv', index=False)